In [1]:
from datetime import datetime
from pprint import pprint
import re
import sqlite3

In [2]:
db = sqlite3.connect('database.sqlite3')

In [3]:
!sqlite3 database.sqlite3 .schema

-- Loading resources from /home/remram/.sqliterc
CREATE TABLE windows(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NOT NULL,
                active BOOLEAN NOT NULL,
                name TEXT NOT NULL
            );
CREATE TABLE runs(
                id INTEGER PRIMARY KEY,
                start DATETIME NOT NULL,
                end DATETIME NULL,
                end_reason TEXT NOT NULL DEFAULT ''
            );
CREATE INDEX idx_windows_start ON windows(start);
CREATE INDEX idx_windows_end ON windows(end);
CREATE INDEX idx_windows_active ON windows(active) WHERE active=1;
CREATE INDEX idx_windows_name ON windows(name);
CREATE INDEX idx_runs_start ON runs(start);
CREATE INDEX idx_runs_end ON runs(end);


In [4]:
list(db.execute('SELECT count(id) FROM windows;'))

[(3032,)]

In [5]:
names = [name for name, in db.execute('SELECT DISTINCT name FROM windows WHERE active=1;')]
len(names)

163

In [6]:
for name in sorted(names):
    print(name)

() datamart-prod — Konsole
() tanis — Konsole
(NSFW) What Fetish did you have until you tried it? : AskReddit — Mozilla Firefox
- : sudo -g — Konsole
- : sudo apt — Konsole
/home/remram/projects/pctracker - QGit
2017 IRONMAN Lake Placid » RTRT.me — Mozilla Firefox
ASMR Cranial Nerve Exam: Concussion Test 👩‍⚕️ (Soft Spoken Roleplay - Doctor Check Up ASMR) - YouTube — Mozilla Firefox
Add Comment | Hacker News — Mozilla Firefox
Amethyst - The open source, data-driven game engine — Mozilla Firefox
Beer distribution game - Wikipedia — Mozilla Firefox
Bevy - A data-driven game engine built in Rust — Mozilla Firefox
Bevy - Apps — Mozilla Firefox
Bevy - Bevy 0.4 — Mozilla Firefox
Bevy - ECS — Mozilla Firefox
Bevy - Faq — Mozilla Firefox
Bevy - Getting Started — Mozilla Firefox
Bevy - Introduction — Mozilla Firefox
Bevy - Plugins — Mozilla Firefox
Bevy - Resources — Mozilla Firefox
Bevy - Setup — Mozilla Firefox
Bevy - Troubleshooting — Mozilla Firefox
Bevy ECS migration by cart · Pull Request 

In [22]:
def prefix(string, name=None):
    def prefix_matcher(record):
        if record.name.startswith(string):
            return record.name[len(string):]
        else:
            return None

    if not name:
        name = repr(pattern)

    return name, prefix_matcher

def suffix(string, name=None):
    def suffix_matcher(record):
        if record.name.endswith(string):
            return record.name[:-len(string)]
        else:
            return None

    if not name:
        name = repr(pattern)

    return name, suffix_matcher

In [23]:
filters = [
    (suffix(' — Mozilla Firefox', 'Firefox'), [
        (suffix(' - YouTube', 'YouTube'), []),
        (suffix(' Hacker News', 'HackerNews'), []),
        (suffix(' GitLab', 'GitLab'), []),
        (suffix(' - Gmail', 'Gmail'), []),
        (prefix('Slack', 'Slack'), []),
        (suffix(' - Jupyter Notebook', 'Jupyter'), []),
    ]),
    (suffix(' - Google Chrome', 'Google Chrome'), []),
    (prefix('Signal', 'Signal'), []),
    (suffix(' - GVIM', 'GVIM'), []),
    (suffix(' — Konsole', 'Konsole'), []),
]

In [24]:
class Record(object):
    def __init__(self, start, end, name):
        self.start = datetime.fromisoformat(start)
        self.end = datetime.fromisoformat(end)
        self.name = name
        
    @property
    def duration(self):
        return (self.end - self.start).total_seconds()

In [25]:
def format_duration(seconds):
    seconds = int(seconds)
    out = '%ds' % (seconds % 60)
    if seconds >= 60:
        out = '%dm%s' % ((seconds // 60) % 60, out)
        if seconds >= 3600:
            out = '%dh%s' % (seconds // 3600, out)
    return out

In [26]:
class OutputNode(object):
    def __init__(self, name):
        self.name = name
        self.duration = 0
        self.children = {}
        
    def child(self, name):
        try:
            child = self.children[name]
        except KeyError:
            child = self.children[name] = OutputNode(name)
        return child
    
    def print(self, indent=0):
        if indent == 0:
            prefix = ''
            last_prefix = '└── '
        else:
            prefix = '│   ' * (indent - 1)
            prefix += '├── '
            last_prefix = '|   ' * indent
            last_prefix += '└── '
        name = self.name or 'total'
        other_duration = self.duration
        duration = format_duration(self.duration)
        print(f'{prefix}{name} {duration}')
        children = sorted(self.children.values(), key=lambda c: -c.duration)
        for child in children:
            child.print(indent + 1)
            other_duration -= child.duration
        if self.children:
            print(f'{last_prefix}other {other_duration}')

In [27]:
def apply_filters(record, filters, output):
    # Add duration to current node
    output.duration += record.duration
    # Apply filters until one matches
    for (name, matcher), then_ops in filters:
        m = matcher(record)
        if m is not None:
            record.name = m
            apply_filters(record, then_ops, output.child(name))
            return


output = OutputNode(None)

for row in db.execute('''\
    SELECT start, end, name
    FROM windows
    WHERE active=1;
'''):
    record = Record(*row)
    apply_filters(record, filters, output)

In [28]:
output.print()

total 4h9m49s
├── Firefox 2h46m58s
│   ├── YouTube 25m37s
│   ├── HackerNews 15m7s
│   ├── Jupyter 12m47s
│   ├── Gmail 10m8s
│   ├── GitLab 5m17s
│   ├── Slack 5m7s
|   └── other 5575.0
├── Konsole 15m17s
├── Signal 3m43s
├── GVIM 1m26s
├── Google Chrome 26s
└── other 3719.0
